In [28]:

import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

import numpy as np
import qutip as qt
from scipy.linalg import expm
import pickle
from qiskit.quantum_info import DensityMatrix, Statevector, partial_trace, state_fidelity
from qiskit import QuantumCircuit

from oft import oft
from tools.classical import find_ideal_heisenberg, trotter_heisenberg_qutip

boltzmann = lambda beta, w: np.min([1, np.exp(-beta * w)])
measured_energy = 0.19634954084936207 #! Always updated for new simulations

In [2]:
num_qubits = 3
num_energy_bits = 6
eps = 0.1
sigma = 5
bohr_bound = 2 ** (-num_energy_bits + 1)
beta = 1
T = 1
total_time = T #! * 2 * np.pi
num_trotter_steps = 10

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)

step_size = total_time / num_trotter_steps
trott_U_qt = trotter_heisenberg_qutip(num_qubits, step_size, num_trotter_steps, 
                                      coeffs=hamiltonian.rescaled_coeffs, shift=(total_time*hamiltonian.shift))

Original spectrum:  [-3.7    -3.177  -2.5482 -1.7     1.625   2.4547  3.2521  3.7936]
Ideal spectrum:  [0.     0.0314 0.0692 0.1201 0.3198 0.3696 0.4175 0.45  ]
Nonrescaled coefficients:  [1.  0.8 0.9 1. ]
Rescaled coefficients:  [0.06005156 0.04804125 0.05404641 0.06005156]
eXX, eYY, eZZ for qubit (0, 1)
eXX, eYY, eZZ for qubit (1, 2)
eXX, eYY, eZZ for qubit (2, 0)


In [8]:
N = 2**num_energy_bits
N_labels = np.arange(N / 2, dtype=int)
N_labels_neg = np.arange(- N / 2, 0, dtype=int)
N_labels = np.concatenate((N_labels, N_labels_neg))
energy_labels = N_labels / N
# pick a random energy
np.random.seed(666)
# w = np.random.choice(energy_labels)
w = measured_energy  #! We dont use the right energy yet
print(f'Energy: {w}')

jump_op = qt.tensor([qt.qeye(2), qt.sigmax(), qt.qeye(2)]).full()

with open(f'/Users/bence/code/liouvillian_metro/data/block_initial_state_n{num_qubits}k{num_energy_bits}.pkl', 'rb') as f:
    initial_state = pickle.load(f)

initial_dm = DensityMatrix(initial_state).data

oft_op = oft(jump_op, w, 2**num_energy_bits, sigma, trotter=trott_U_qt.full())
dm_we_should_get = boltzmann(beta, w)*(oft_op @ initial_dm @ oft_op.conj().T)
    
dm_we_should_get /= np.trace(dm_we_should_get)


Energy: 0.19634954084936207


In [38]:
def energy_state_from_bits(bitsring: str) -> np.ndarray:
    """Convert a bitstring to an energy eigenstate."""
    reversed_bitstring = bitsring[::-1]
    tensor_list = []
    for bit in reversed_bitstring:
        if bit == '0':
            tensor_list.append(qt.basis(2, 0))
        else:
            tensor_list.append(qt.basis(2, 1))
    state = qt.tensor(tensor_list).full()
    return state / np.linalg.norm(state)

ex_bitstring = '10'
energy_state = energy_state_from_bits(ex_bitstring)
print(energy_state)
circ = QuantumCircuit(2)
circ.x(0)
st = Statevector(circ).data

[[0.+0.j]
 [1.+0.j]
 [0.+0.j]
 [0.+0.j]]


array([1.+0.j])

In [19]:
with open(f'/Users/bence/code/liouvillian_metro/data/block_state_before_measure_n{num_qubits}k{num_energy_bits}.pkl', 'rb') as f:
    circ_state = pickle.load(f)

projector_to_boltz_zero = qt.tensor([qt.qeye(2**(num_qubits + num_energy_bits)), qt.basis(2, 0).dag()]).full()
projected_circ_state = projector_to_boltz_zero @ circ_state.data
projected_circ_state /= np.linalg.norm(projected_circ_state)

projected_circ_sys_dm = partial_trace(projected_circ_state, list(range(num_energy_bits))).data
projected_circ_sys_dm /= np.trace(projected_circ_sys_dm)


(512,)
(8, 8)


In [21]:
dist = np.linalg.norm(dm_we_should_get - projected_circ_sys_dm)
dist

0.5708647702659243